# White House visitor logs

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import datetime as dt
import altair as alt
import numpy as np

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.options.display.max_colwidth = None

### Data dictionary

In [4]:
# "https://obamawhitehouse.archives.gov/files/disclosures/visitors/WhiteHouse-WAVES-Key-1209.txt"

In [5]:
today = dt.date.today().strftime("%m-%d-%Y")

In [6]:
df = pd.read_csv(
    "data/processed/log_latest.csv",
    parse_dates=[
        "appt_made_date",
        "appt_start_date",
        "appt_end_date",
        "appt_cancel_date",
        "releasedate",
    ],
)

In [7]:
df.sort_values(["appt_made_date"], ascending=False).head()

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,releasedate
8035,VARGAS,ANDY,N,U30780,NaN,VA,NAN,NAN,NAN,NAN,2021-08-31 06:30:00,2021-08-31 09:00:00,2021-08-31 23:59:00,NaT,1.0,KG,WIN,8/31/21 6:30,KG,NAN,PLAYMAKER,VPR,RESIDENCE,GOODHUE,KEVIN,NaN,NAN,2021-12-03
8019,LASKOWSKI,ELLISON,S,U30721,177915.0,VA,8/30/21 14:57,NAN,NAN,NAN,2021-08-30 11:12:00,2021-08-30 00:00:00,2021-08-30 11:59:00,NaT,1.0,PD,WIN,8/30/21 11:12,PD,DIEN,PETER,OEOB,389,DIEN,PETER,NaN,NAN,2021-12-03
8020,LESKO,MARK,J,U30720,183048.0,VA,8/30/21 12:43,NAN,NAN,B0401,2021-08-30 10:59:00,2021-08-30 00:00:00,2021-08-30 11:59:00,NaT,4.0,KR,WIN,8/30/21 10:59,KR,REMUS,DANA,WH,WW202,REILLY,KATHARINE,NaN,NAN,2021-12-03
8018,KOHLER,ALAN,E,U30720,183572.0,VA,8/30/21 12:30,NAN,NAN,NAN,2021-08-30 10:35:00,2021-08-30 00:00:00,2021-08-30 11:59:00,NaT,4.0,KR,WIN,8/30/21 10:35,KR,REMUS,DANA,WH,WW202,REILLY,KATHARINE,NaN,NAN,2021-12-03
7998,ARNOLD,NIKKI,F,U30720,183104.0,VA,8/30/21 12:30,NAN,NAN,B04,2021-08-30 10:35:00,2021-08-30 00:00:00,2021-08-30 11:59:00,NaT,4.0,KR,WIN,8/30/21 10:35,KR,REMUS,DANA,WH,WW202,REILLY,KATHARINE,NaN,NAN,2021-12-03


### Buildings

In [8]:
df.meeting_loc.value_counts()

OEOB    7056
WH      6559
NEOB     851
EEOB     837
VPR      492
Name: meeting_loc, dtype: int64

### Rooms 

In [9]:
df.meeting_room.value_counts().head()

SOUTH LAWN    1253
EW206          884
445            760
RES - STAT     721
374            420
Name: meeting_room, dtype: int64

### Who's visited the Oval?

In [10]:
len(df[df["meeting_room"].str.contains("OVAL")])

320

In [11]:
df[df["meeting_room"].str.contains("OVAL")].head()

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,releasedate
115,ESHOO,ANNA,G,U21725,NaN,VA,3/3/21 13:31,B0401,NAN,NAN,2021-03-03 11:27:00,2021-03-03 13:30:00,2021-03-03 23:59:00,NaT,9.0,GT,WIN,3/3/2021,GT,NAN,POTUS,WH,OVAL OFFIC,TAYLOR,GABRIELLE,NaN,NAN,2021-06-29
116,VANHOLLEN,CHRISTOPHER,N,U21725,NaN,VA,3/3/21 13:27,B0401,NAN,NAN,2021-03-03 11:27:00,2021-03-03 13:30:00,2021-03-03 23:59:00,NaT,9.0,GT,WIN,3/3/2021,GT,NAN,POTUS,WH,OVAL OFFIC,TAYLOR,GABRIELLE,NaN,NAN,2021-06-29
117,DEGETTE,DIANA,L,U21725,NaN,VA,NAN,NAN,NAN,NAN,2021-03-03 11:27:00,2021-03-03 13:30:00,2021-03-03 23:59:00,NaT,9.0,GT,WIN,3/3/2021,GT,NAN,POTUS,WH,OVAL OFFIC,TAYLOR,GABRIELLE,NaN,NAN,2021-06-29
118,UPTON,FREDERICK,S,U21725,NaN,VA,NAN,NAN,NAN,NAN,2021-03-03 11:27:00,2021-03-03 13:30:00,2021-03-03 23:59:00,NaT,9.0,GT,WIN,3/3/2021,GT,NAN,POTUS,WH,OVAL OFFIC,TAYLOR,GABRIELLE,NaN,NAN,2021-06-29
121,CRAPO,MICHAEL,D,U21725,NaN,VA,NAN,NAN,NAN,NAN,2021-03-03 11:27:00,2021-03-03 13:30:00,2021-03-03 23:59:00,NaT,9.0,GT,WIN,3/3/2021,GT,NAN,POTUS,WH,OVAL OFFIC,TAYLOR,GABRIELLE,NaN,NAN,2021-06-29


### Where meeting room is ...

In [12]:
df[df["meeting_room"].str.contains("EW206")].head()

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,releasedate
288,MENDEZ,ANTHONY,A,U21895,NaN,VA,NAN,NAN,NAN,NAN,2021-03-08 08:41:00,2021-03-08 10:30:00,2021-03-08 23:59:00,NaT,5.0,DV,WIN,3/8/2021,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,NAN,2021-06-29
290,CAMPBELL,EVAN,M,U21895,NaN,VA,NAN,NAN,NAN,NAN,2021-03-08 08:41:00,2021-03-08 10:30:00,2021-03-08 23:59:00,NaT,5.0,DV,WIN,3/8/2021,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,NAN,2021-06-29
291,KLIMCZAK,FRANCES,D,U21895,NaN,VA,NAN,NAN,NAN,NAN,2021-03-08 08:41:00,2021-03-08 10:30:00,2021-03-08 23:59:00,NaT,5.0,DV,WIN,3/8/2021,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,NAN,2021-06-29
292,AHRENS,KATHERINE,L,U21895,NaN,VA,NAN,NAN,NAN,NAN,2021-03-08 08:41:00,2021-03-08 10:30:00,2021-03-08 23:59:00,NaT,5.0,DV,WIN,3/8/2021,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,NAN,2021-06-29
440,REGINA,ANTHONY,C,U22043,NaN,VA,3/11/21 13:02,SOS,NAN,NAN,2021-03-10 11:34:00,2021-03-11 13:00:00,2021-03-11 23:59:00,NaT,22.0,DV,WIN,3/10/2021,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,NAN,2021-06-29


### Where 'visitee' is explicity the Vice President

In [13]:
len(df[df["visitee_namefirst"].str.contains("VPOTUS")])

45

In [14]:
df[df["visitee_namefirst"].str.contains("VPOTUS")].to_csv(
    "data/processed/vpotus_visitee.csv", index=False
)